# Optimizer builder

In [1]:
from ipynb.fs.full.Rectangle import Rectangle
from ipynb.fs.full.Rectangle import pick_random_sol, get_border_search_field
from ipynb.fs.full.Rectangle import polygon_bounding
from ipynb.fs.full.Rectangle import log_polygons, erase_logs, LOGS, create_animation

In [2]:
from ipynb.fs.full.PSO import PSO
from ipynb.fs.full.DE import DE

In [10]:
class OptimizerBuilder:
    """Builder for PSO & DE optimizers"""
    
    PSO_label = "PSO"
    DE_label = "DE"
    
    inf, sup = None, None
    eval_func = None
    bounding_func = None
    init_func = None 
    log_func = None
    
    default_colors = ["green", "red"]
        
    def __init__(self, land,
                 accept_when_invalid_move = False,
                 dim=5, 
                 n_agents=30, 
                 n_cycles=500,
                 log_padding=50,
                 isbetter_func=lambda f1, f2: f1 > f2,
                 show_logs=False):
        self.accept_when_invalid_move = accept_when_invalid_move
        self.dim = dim
        self.n_agents = n_agents
        self.n_cycles = n_cycles
        self.log_padding = log_padding
        self.isbetter_func = isbetter_func
        self.show_logs = show_logs
        self.set_land(land)
        
    def generate_methods(self):
        if self.accept_when_invalid_move:
            self.eval_func = lambda x: (
                rect := Rectangle(*x),
                rect.area() if rect.is_valid(self.land) else -rect.area())[-1]
            self.bounding_func = lambda x: x
        else:
            self.eval_func = lambda x: Rectangle(*x).area()
            self.bounding_func = lambda x: polygon_bounding(x, self.land)

        self.log_func = lambda x, it=0, fitness=None, legend=None, text=None, padding=self.log_padding, colors=self.default_colors: (
            log_polygons(self.land, x, colors, legend, text, padding, it, fitness, show=self.show_logs))

        self.init_func = lambda: pick_random_sol(self.land, self.n_agents, inf=self.inf, sup=self.sup)

    def set_land(self, land):
        self.land = land
        self.inf, self.sup = get_border_search_field(land)
        self.generate_methods()
        
    def build_PSO(self, PSI=0.7, CMAX=1.47):
        return PSO(eval_func=self.eval_func, 
                bounding_func=self.bounding_func, 
                log_func=self.log_func,
                init_func=self.init_func,
                isbetter_func=self.isbetter_func,
                n_agents=self.n_agents, 
                n_cycles=self.n_cycles,
                dim=self.dim,
                PSI=PSI,
                CMAX=CMAX)

    def build_DE(self, CR=0.9, F=0.8):
        return DE(eval_func=self.eval_func, 
                bounding_func=self.bounding_func, 
                log_func=self.log_func,
                init_func=self.init_func,
                isbetter_func=self.isbetter_func,
                n_agents=self.n_agents, 
                n_cycles=self.n_cycles,
                dim=self.dim,
                CR=CR,
                F=F)
    def build(self, optimizer_label, *args):
        if optimizer_label == self.PSO_label:
            return self.build_PSO(*args)
        else:
            return self.build_DE(*args)

In [9]:
if __name__ == '__main__':
    land = [(0,0), (0,100), (100,100), (110, 50), (100, 0)]
    builder = OptimizerBuilder(land=land, 
                               accept_when_invalid_move=False, 
                               n_agents=100, 
                               n_cycles=1000)
    pso_optimizer = builder.build_PSO(0.17, 1.47)
    erase_logs("PSO")
    pso_optimizer.fit()
    display(create_animation(LOGS["PSO - Best"], land, "Best"))

100%|██████████| 1000/1000 [00:04<00:00, 203.65it/s]
